## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [2]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_27.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_27.csv')

DI_train = pd.read_csv('../data/DI_train_dataset_27.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_27.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [4]:
print(f"IVF_X shape: {IVF_X.shape}")
print(f"IVF_test shape: {IVF_test.drop('ID', axis=1).shape}")
print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

IVF_X shape: (250052, 85)
IVF_test shape: (87891, 85)
DI_X shape: (6289, 36)
DI_test shape: (2176, 36)


### 인코딩 

In [5]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [6]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "정자_기증자_나이"
]

In [7]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [8]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

### DI 데이터

In [9]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'max_depth': trial.suggest_int('max_depth', 3, 300),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': 42,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1   
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=4000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-12 13:07:15,933] A new study created in memory with name: no-name-c63dc0b3-1564-45ae-be64-0853c088c3c0
[I 2025-02-12 13:07:17,776] Trial 0 finished with value: 0.7004108938371647 and parameters: {'n_estimators': 2817, 'max_depth': 229, 'learning_rate': 0.002587311190209361, 'num_leaves': 1724, 'min_child_samples': 26, 'subsample': 0.7901579390606747, 'reg_alpha': 4.006401946359089, 'reg_lambda': 0.09637474738626133}. Best is trial 0 with value: 0.7004108938371647.
[I 2025-02-12 13:07:19,647] Trial 1 finished with value: 0.6814886031326363 and parameters: {'n_estimators': 1897, 'max_depth': 238, 'learning_rate': 0.0034182930717089983, 'num_leaves': 1912, 'min_child_samples': 61, 'subsample': 0.8490071066663631, 'reg_alpha': 0.04328353180466825, 'reg_lambda': 0.18093020755008069}. Best is trial 0 with value: 0.7004108938371647.
[I 2025-02-12 13:07:20,275] Trial 2 finished with value: 0.6968427558500867 and parameters: {'n_estimators': 1995, 'max_depth': 192, 'learning_rate': 0

Best trial:
  Value: 0.7246067036356666
  Params: 
    n_estimators: 4604
    max_depth: 276
    learning_rate: 0.09268921690273974
    num_leaves: 2
    min_child_samples: 70
    subsample: 0.603490858727672
    reg_alpha: 0.010665492916529533
    reg_lambda: 0.0022411835420855


.